In [55]:
import numpy as np
import pandas as pd

In [2]:
# import data
logfile = open("../../Data/Logs/light1_first_se_log.txt")
# logfile = open("../../Data/Logs/Light1_OperationalTest_May6_ScriptLog_1.txt")
loglines = logfile.readlines()
print (loglines)

['# Script started\n', 'SE0>rtc read\n', 'curr rtc time 1646297867.353394 s, 2022-03-03 08:57:47 \n', 'OK\n', 'SE0># Script delay (avoiding SAA)\n', 'SE0>script delayuntil 1646297935\n', 'OK\n', 'SE0># Enabling the PC and payload (and switching off with the timer of 30m for the payload and 3h for the PC)\n', 'SE0>csp txrx 4 14 600 000500302A\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0007000807\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0008000807\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0005010100\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0007011400\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp t

In [3]:
for i in range(len(loglines)): # i is a number for every line in the log file
    k = loglines[i] 
    if "SE0>rtc read" in k:
        I = i+1
        l = loglines[I][14:]
        print ("Line:" , i , ", Command:" , k , ", Time:" , l )

# if the value is in range of the full range of file?
# first rtc must be start of code, second rtc must be end of code

Line: 1 , Command: SE0>rtc read
 , Time: 1646297867.353394 s, 2022-03-03 08:57:47 

Line: 139 , Command: SE0>rtc read
 , Time: 1646299664.765045 s, 2022-03-03 09:27:44 



In [4]:
for i in range(len(loglines)): # i is a number for every line in the log file
    k = loglines[i] 
    # l = loglines[i+1]
    if k[0:12] in "SE0>rtc read":
        # l=i+1
        print ("line: " , i , ", command: " , k )




    # if k[0:5] in "SE0>#":
    #     print (k)

line:  1 , command:  SE0>rtc read

line:  139 , command:  SE0>rtc read

line:  142 , command:  SE0>


In [5]:
# Work to find the following:

# "Time,scenario"
# "At time () command () was excuted.
# "At time () command () failed."

# If line in SE0>:
# print the time
# print the command
# print the info
# print if command was excuted or failed

In [80]:
cmdlist = pd.read_csv("command_list.csv")
type(cmdlist)

pandas.core.frame.DataFrame

In [104]:
line = 'SE0>csp txrx 12 8 3000 08'
splt = line.split(' ')
print(splt)
if 'txrx' in splt[1]:
    print('this is a command')
    node,port,msg = splt[2],splt[3],splt[5]
    index = cmdlist.loc[(cmdlist['NODE']==12) & (cmdlist['PORT']==port) & (cmdlist['Message']==msg),['Description']]
    print(index)
    if len(index) == 0:
        print('Command not found in command list: ', line)
    else:
        print(index)

elif splt[1] == 'delay':
    print('this is a delay')

elif splt[1] == 'delayuntil':
    print('this is a delayuntil')

elif splt[1] == 'read':
    print('read the current time')




['SE0>csp', 'txrx', '12', '8', '3000', '08']
this is a command
Empty DataFrame
Columns: [Description]
Index: []
Command not found in command list:  SE0>csp txrx 12 8 3000 08


In [ ]:
def desc_time_finder(line:str):
    # command,node,port,message
    splt = line.split(' ')

In [47]:
def decode_log(filename:str="../../Data/Logs/light1-SD-1016-se-log.txt"):
    # Load the logfile
    logfile = open(filename)
    cmdlist = pd.read_csv("command_list.csv")

    # Load the lines
    loglines = logfile.readlines()

    # Close the file
    logfile.close()

    # Create an array with the lines
    commands    = []
    outputs     = []
    idx         = []
    description = []

    # time = 0

    # Get commands and their indices
    for i, line in enumerate(loglines):
        if 'SE0>' in line:
            commands.append(line)
            idx.append(i)
            description_finder(line)

    # Check if the last command was empty
    if loglines[-1] != 'SE0>':
        commands.append('SE0>')
        idx.append(len(loglines))

    # Get the command output
    for i in range(len(idx)-1):
        out = []
        if 'SE0>#' not in commands[i]:
            for j in range(idx[i]+1,idx[i+1]):
                out.append(loglines[j])
        
        outputs.append(out)

    # Create the dictionary
    log = [[command,output] for command,output in zip(commands,outputs)]

    # Create a dictionary
    return log, commands[:-1], outputs, description

In [48]:
# Get hte log file and the commands
log, commands, outputs   = decode_log()

In [50]:
# Check which commands executed correcly
failed_idx = []
failed_cmd = []
failed_output = []

# Find the commands that failed
for i in range(len(commands)):
    for output in log[i][1]:
        if 'FAIL' in output:
            failed_idx.append(i)
            failed_cmd.append(commands[i])
            failed_output.append(outputs[i])

In [54]:
# print (failed_idx)
# print (failed_cmd,failed_output)
# print (failed_output)

In [ ]:
# Decode the file

In [ ]:
# Define the command description in a dict
# return timestamps of the scenarios and the failures (use the delays and the rtc reads to find the time)
# id,line,command,description,output


In [ ]:
# line = SE0>csp txrx 12 8 3000 08
def f(line:str):
    line[13:15]